## Hackaton Code

In [41]:
import random
import os
import pandas as pd
from pandas import read_csv
import numpy as np
import re
from os.path import isfile, join
from sklearn.datasets import load_iris
import glob

class Person(object):
    def __init__(self):
        self.ID = 0
        self.age = 0
        self.status = "Susceptible"
        self.days_sick = 0
        self.max_days_sick = 5
        self.interactions = []
        self.mask = False #Add mask parameter
        self.washed_hands = False # Add wash hands parameter
        self.social_dist = False #Add social distancing parameter
        
    def set_infection_rate(self, new_rate):
        self.infection_rate = new_rate
    
    def infect(self, number_of_days):
        self.days_sick = number_of_days
        self.status = "Infected"
        
    def recover(self):
        self.days_sick = 0
        self.status = "Recovered"
    
    # Change function with new infection rate
    def roll_for_infection(self,infection_rate):
        if (random.randint(0,101) < infection_rate):
            self.infect(self.max_days_sick)
            
    def get_status(self):
        return self.status
    
    def get_days_sick(self):
        return self.days_sick
    # Getters Added
    
    def get_mask(self):
        return self.mask
    def get_hands(self):
        return self.washed_hands
    def get_social_dist(self):
        return self.social_dist
    # Setters Added
    def set_mask(self,boolean):
        self.mask = boolean
    def set_hands(self,boolean):
        self.washed_hands = boolean
    def set_social_dist(self,boolean):
        self.social_dist = boolean
    
    #Get prevention flag
    def get_full_prevention(self):
        # 1: Mask - 2: Hands - 3: Social Dist
        return [self.mask, self.washed_hands, self.social_dist]
    
    def update(self):
        if (self.status == "Susceptible"):
            self.roll_for_infection()
        elif (self.status == "Infected"):
            self.days_sick = self.days_sick - 1
            if (self.days_sick <= 0):
                self.recover()
        

In [43]:
class Population(object):
    
    def __init__(self, number_of_people=1000, default_interactions=10, default_days_sick=5, default_infection_rate=10):
        
        self.People = []
        self.population_count = number_of_people
        self.number_of_interactions = default_interactions
        self.max_days_sick = default_days_sick
        self.infection_rate = default_infection_rate
        #Add list for patients with non preventions methods
        self.number_of_people_with_non_prev = 0
        #Add list for patients with three preventions methods
        self.number_of_people_with_all_prev = 0
        #Add list for patients that wash their hands and use mask
        self.number_of_people_hands_and_mask = 0
        #Add list for patients that wash their hands and social dist
        self.number_of_people_hands_and_social = 0
        #Add list for patients that social dist and use mask
        self.number_of_people_social_and_mask = 0
        self.number_of_people_only_hands_by_day = 0
        self.number_of_people_only_mask_by_day = 0
        self.number_of_people_only_social_dist_by_day = 0
        
        ## set Patient_Zero aka The Jerk
        patient_zero = Person()
        patient_zero.infect(self.max_days_sick)
        
        ## add patient_zero to our People[]
        
        self.People.append(patient_zero)
        
        ## build the rest of our Population that use prevention methods
        for i in range(0, self.population_count):
            # Add new person
            new_person = Person()
            # Randoms for prevention methods
            mask_random = random.randrange(0,101)
            hands_random = random.randrange(0,101)
            social_dist_random = random.randrange(0,101)
            # if the random number is a pair then it changes the flag
            if (mask_random%2==0):
                new_person.set_mask(True)
            if (hands_random%2==0):
                new_person.set_hands(True)
            if (social_dist_random%2==0):
                new_person.set_social_dist(True)
            #Adds the person into the population list
            self.People.append(new_person)
            
    
    ##go through our population, and count each person that's susceptible
    def number_of_susceptible(self):
        count = 0
        
        for the_person in self.People:
            if (the_person.get_status() == "Susceptible"):
                count = count + 1
        
        return count
    ## go through our population, and count each person that's wearing a mask
    def number_of_masks(self):
        count = 0
        
        for the_person in self.People:
            if (the_person.get_mask()):
                count = count + 1
        
        return count
    ##go through the population, and count each person that's infected
    def number_of_infected(self):
        count = 0
        
        for the_person in self.People:
            if (the_person.get_status() == "Infected"):
                count = count + 1
        
        return count
    
    ##go through the population, and count each person that's recovered
    def number_of_recovered(self):
        count = 0
        
        for the_person in self.People:
            if (the_person.get_status() == "Recovered"):
                count = count + 1
                
        return count
    
    ##Total Population
    def get_population_count(self):
        return self.People.count()
    ## Total of people that wash their hands
    def number_of_washed_hands(self):
        count = 0
        
        for the_person in self.People:
            if (the_person.get_hands()):
                count = count + 1
        
        return count
    ## Total of people that who social dist
    def number_of_soial_dist(self):
        count = 0
        for the_person in self.People:
            if (the_person.get_social_dist()):
                count = count + 1
        
        return count
    
    def set_default_lists(self):
        #Add list for patients with non preventions methods
        self.number_of_people_with_non_prev = 0
        #Add list for patients with three preventions methods
        self.number_of_people_with_all_prev = 0
        #Add list for patients that wash their hands and use mask
        self.number_of_people_hands_and_mask = 0
        #Add list for patients that wash their hands and social dist
        self.number_of_people_hands_and_social = 0
        #Add list for patients that social dist and use mask
        self.number_of_people_social_and_mask = 0
        self.number_of_people_only_hands_by_day = 0
        self.number_of_people_only_mask_by_day = 0
        self.number_of_people_only_social_dist_by_day = 0
        
    ## Get the data lists
    def set_data_lists(self,status):
        self.set_default_lists()
        for person in self.People:
            if (person.get_status() == status):
                
                if (sum(person.get_full_prevention()) == 3):
                    self.number_of_people_with_all_prev = self.number_of_people_with_all_prev + 1
                    
                elif (sum(person.get_full_prevention()) == 0):
                    self.number_of_people_with_non_prev = self.number_of_people_with_non_prev + 1
                    
                elif (sum(person.get_full_prevention()) == 1):
                    if (person.mask):
                       #Add to list for patients that use mask
                        self.number_of_people_only_mask_by_day = self.number_of_people_only_mask_by_day + 1
                    if (person.washed_hands):
                        #Add to list for patients that wash their hands
                        self.number_of_people_only_hands_by_day = self.number_of_people_only_hands_by_day + 1
                    if (person.social_dist):
                        #Add to  list for patients that social dist
                        self.number_of_people_only_social_dist_by_day = self.number_of_people_only_social_dist_by_day + 1
                        
                else:
                    if (person.mask and person.washed_hands):
                       #Add to list for patients that wash their hands and use mask
                        self.number_of_people_hands_and_mask = self.number_of_people_hands_and_mask + 1
                    if (person.washed_hands and person.social_dist):
                        #Add to list for patients that wash their hands and social dist
                        self.number_of_people_hands_and_social = self.number_of_people_hands_and_social + 1
                    if (person.social_dist and person.mask):
                        #Add to  list for patients that social dist and use mask
                        self.number_of_people_social_and_mask = self.number_of_people_social_and_mask + 1
    
    ## Pirnt data
    def print_data(self,status):
        if (status == "Infected"):
            print("\n---------Data of the infected population---------")
            print("\nPeople with no preventions: ", self.number_of_people_with_non_prev)
            print("\nPeople with all preventions: ", self.number_of_people_with_all_prev)
            print("\nPeople that wear masks and wash their hands: ", self.number_of_people_hands_and_mask)
            print("\nPeople that wear masks and do social distance: ", self.number_of_people_hands_and_social)
            print("\nPeople that wash their hands and do social distance: ", self.number_of_people_social_and_mask)
            print("------------------------------------------------------\n")
        elif (status == "Recovered"):
            print("\n---------Data of the recoverdd population---------")
            print("\nPeople with no preventions: ", self.number_of_people_with_non_prev)
            print("\nPeople with all preventions: ", self.number_of_people_with_all_prev)
            print("\nPeople that wear masks and wash their hands: ", self.number_of_people_hands_and_mask)
            print("\nPeople that wear masks and do social distance: ", self.number_of_people_hands_and_social)
            print("\nPeople that wash their hands and do social distance: ", self.number_of_people_social_and_mask)
            print("------------------------------------------------------\n")
        else:
            print("\n---------Data of the susceptible population---------")
            print("\nPeople with no preventions: ", self.number_of_people_with_non_prev)
            print("\nPeople with all preventions: ", self.number_of_people_with_all_prev)
            print("\nPeople that wear masks and wash their hands: ", self.number_of_people_hands_and_mask)
            print("\nPeople that wear masks and do social distance: ", self.number_of_people_hands_and_social)
            print("\nPeople that wash their hands and do social distance: ", self.number_of_people_social_and_mask)
            print("------------------------------------------------------\n")
            
    def update(self):
        
        ##we're going to make a list of People who might get
        ##infected at the end of the day
        
        might_get_sick = []
        
        ## go through my Population and see who might get sick
        for person in self.People:
            person.interactions = []
            
            ## each person inteteracts w/ number_of_interaction number of people
            
            for i in range(0, self.number_of_interactions):
                r = random.randint(0, self.population_count)
                person.interactions.append(r)
            
            ## if person is infected, their interactions might get infected
            if (person.get_status() == "Infected"):
                ##????? really????
                person.update()
                
                for j in person.interactions:
                    if (self.People[j].get_status() == "Susceptible"):
                        might_get_sick.append(self.People[j])
            
            ## if person is susceptible, they interact w/ someone infected
            if (person.get_status() == "Susceptible"):
                for j in person.interactions:
                    if (self.People[j].get_status() == "Infected"):
                        might_get_sick.append(person)
                        
        ##the above updates each person who is infected
        ##I have a list of people who **might** get infected
        ##each person in that list, has to roll to get infected
        
        for person in might_get_sick:
            #Add new if to check if the person has a all the prevention methods
            list_of_preventions = person.get_full_prevention()
            if (sum(list_of_preventions) > 0):
                # Set the infection rate for the person with each prevention methods
                person_infection_rate = self.infection_rate #10
                if (person.mask): 
                    person_infection_rate =  person_infection_rate - (person_infection_rate*0.4)
                if (person.washed_hands): 
                    person_infection_rate =  person_infection_rate - (person_infection_rate*0.3)
                if (person.social_dist): 
                    person_infection_rate =  person_infection_rate - (person_infection_rate*0.2)
                    
                person.roll_for_infection(person_infection_rate)
            else:
                # 10
                person.roll_for_infection(self.infection_rate)

In [44]:
##Build our Community

myCommunity = Population(40001)
myCommunity.set_data_lists("Susceptible")
day = 0
number_infected = myCommunity.number_of_infected()
print("----- Initial Data -----")
myCommunity.print_data("Susceptible")
print("---------------------------")
# ----------- Day -----------
days = []
# ----------- Susceptible -----------
list_of_susceptible_by_day_sus = []
list_of_infected_by_day_sus = []
list_of_recovered_by_day_sus = []
list_of_all_preventions_by_day_sus = []
list_of_non_preventions_by_day_sus = []
people_only_hands_by_day_sus = []
people_only_mask_by_day_sus = []
people_only_social_dist_by_day_sus = []
people_hands_and_mask_by_day_sus = []
people_hands_and_social_by_day_sus = []
people_social_and_mask_by_day_sus = []
# ----------- Infected -----------
list_of_susceptible_by_day_inf = []
list_of_infected_by_day_inf = []
list_of_recovered_by_day_inf = []
list_of_all_preventions_by_day_inf = []
list_of_non_preventions_by_day_inf = []
people_only_hands_by_day_inf = []
people_only_mask_by_day_inf = []
people_only_social_dist_by_day_inf = []
people_hands_and_mask_by_day_inf = []
people_hands_and_social_by_day_inf = []
people_social_and_mask_by_day_inf = []
# ----------- Recovered -----------
list_of_susceptible_by_day_rec_rec = []
list_of_infected_by_day_rec = []
list_of_recovered_by_day_rec = []
list_of_all_preventions_by_day_rec = []
list_of_non_preventions_by_day_rec = []
people_only_hands_by_day_rec = []
people_only_mask_by_day_rec = []
people_only_social_dist_by_day_rec = []
people_hands_and_mask_by_day_rec = []
people_hands_and_social_by_day_rec = []
people_social_and_mask_by_day_rec = []

def create_dataframe(status, day, list_of_susceptible_by_day, list_of_infected_by_day, list_of_recovered_by_day, list_of_all_preventions_by_day, list_of_non_preventions_by_day,
                     people_hands_and_mask_by_day, people_hands_and_social_by_day, people_social_and_mask_by_day, people_only_hands_by_day, people_only_mask_by_day, people_only_social_dist_by_day):
    #Columns names
    cols = ["Day","Susceptible", "Infected", "Recovered","All Preventions", "No Preventions", "Only Hands", "Only Mask", "Only Social Dist",
            "Hands and Mask", "Hands and Social Dist", "Mask and Social Dist"]
    #List for datas
    df_list = []
    
    data = [day, list_of_susceptible_by_day, list_of_infected_by_day, list_of_recovered_by_day, list_of_all_preventions_by_day, list_of_non_preventions_by_day, people_only_hands_by_day, 
            people_only_mask_by_day, people_only_social_dist_by_day, people_hands_and_mask_by_day, people_hands_and_social_by_day, people_social_and_mask_by_day]
    
    #For that iterates each model
    for i in range(0,len(data)):
        #Creat tmp rank dataframe
        df = pd.DataFrame(data[i], columns=[cols[i]])
        # Add df to the list
        df_list.append(df) 
        
    # Concant dfs
    df_final = pd.concat(df_list, axis=1) # Add ranks and create a new df
    print("Done! " + status + "!")
    df_final.to_csv(status + ".csv", index = False, header=True)
    return df_final
while (number_infected > 0):
    print("Day #", day)
    days.append(day)
    number_infected = myCommunity.number_of_infected()
    number_recovered = myCommunity.number_of_recovered()
    print("Infected: ", number_infected, "Recovered: ", number_recovered)
    #myCommunity.print_data("Susceptible")
    # ----------- Susceptible -----------
    myCommunity.set_data_lists("Susceptible")
    list_of_infected_by_day_sus.append(number_infected)
    list_of_recovered_by_day_sus.append(number_recovered)
    list_of_susceptible_by_day_sus.append(myCommunity.number_of_susceptible())
    list_of_non_preventions_by_day_sus.append(myCommunity.number_of_people_with_non_prev)
    list_of_all_preventions_by_day_sus.append(myCommunity.number_of_people_with_all_prev)
    people_hands_and_mask_by_day_sus.append(myCommunity.number_of_people_hands_and_mask)
    people_hands_and_social_by_day_sus.append(myCommunity.number_of_people_hands_and_social)
    people_social_and_mask_by_day_sus.append(myCommunity.number_of_people_social_and_mask)
    people_only_hands_by_day_sus.append(myCommunity.number_of_people_only_hands_by_day)
    people_only_mask_by_day_sus.append(myCommunity.number_of_people_only_mask_by_day)
    people_only_social_dist_by_day_sus.append(myCommunity.number_of_people_only_social_dist_by_day)
    # ----------- Infected -----------
    #myCommunity.print_data("Infected")
    myCommunity.set_data_lists("Infected")
    list_of_infected_by_day_inf.append(number_infected)
    list_of_recovered_by_day_inf.append(number_recovered)
    list_of_susceptible_by_day_inf.append(myCommunity.number_of_susceptible())
    list_of_non_preventions_by_day_inf.append(myCommunity.number_of_people_with_non_prev)
    list_of_all_preventions_by_day_inf.append(myCommunity.number_of_people_with_all_prev)
    people_hands_and_mask_by_day_inf.append(myCommunity.number_of_people_hands_and_mask)
    people_hands_and_social_by_day_inf.append(myCommunity.number_of_people_hands_and_social)
    people_social_and_mask_by_day_inf.append(myCommunity.number_of_people_social_and_mask)
    people_only_hands_by_day_inf.append(myCommunity.number_of_people_only_hands_by_day)
    people_only_mask_by_day_inf.append(myCommunity.number_of_people_only_mask_by_day)
    people_only_social_dist_by_day_inf.append(myCommunity.number_of_people_only_social_dist_by_day)
    # ----------- Recovered -----------
    #myCommunity.print_data("Recovered")
    myCommunity.set_data_lists("Recovered")
    list_of_infected_by_day_rec.append(number_infected)
    list_of_recovered_by_day_rec.append(number_recovered)
    list_of_susceptible_by_day_rec.append(myCommunity.number_of_susceptible())
    list_of_non_preventions_by_day_rec.append(myCommunity.number_of_people_with_non_prev)
    list_of_all_preventions_by_day_rec.append(myCommunity.number_of_people_with_all_prev)
    people_hands_and_mask_by_day_rec.append(myCommunity.number_of_people_hands_and_mask)
    people_hands_and_social_by_day_rec.append(myCommunity.number_of_people_hands_and_social)
    people_social_and_mask_by_day_rec.append( myCommunity.number_of_people_social_and_mask)
    people_only_hands_by_day_rec.append( myCommunity.number_of_people_only_hands_by_day)
    people_only_mask_by_day_rec.append( myCommunity.number_of_people_only_mask_by_day)
    people_only_social_dist_by_day_rec.append(myCommunity.number_of_people_only_social_dist_by_day)
    
    day = day + 1
    myCommunity.update()

create_dataframe("Infected", days, list_of_susceptible_by_day_inf, list_of_infected_by_day_inf, list_of_recovered_by_day_inf, list_of_all_preventions_by_day_inf, list_of_non_preventions_by_day_inf,
                people_hands_and_mask_by_day_inf, people_hands_and_social_by_day_inf, people_social_and_mask_by_day_inf, people_only_hands_by_day_inf, people_only_mask_by_day_inf, 
                people_only_social_dist_by_day_sus)




----- Initial Data -----

---------Data of the susceptible population---------

People with no preventions:  4894

People with all preventions:  5218

People that wear masks and wash their hands:  5046

People that wear masks and do social distance:  5062

People that wash their hands and do social distance:  5030
------------------------------------------------------

---------------------------
Day # 0
Infected:  1 Recovered:  0
Day # 1
Infected:  2 Recovered:  0
Day # 2
Infected:  5 Recovered:  0
Day # 3
Infected:  10 Recovered:  0
Day # 4
Infected:  25 Recovered:  0
Day # 5
Infected:  53 Recovered:  1
Day # 6
Infected:  122 Recovered:  2
Day # 7
Infected:  276 Recovered:  5
Day # 8
Infected:  589 Recovered:  10
Day # 9
Infected:  1245 Recovered:  25
Day # 10
Infected:  2666 Recovered:  54
Day # 11
Infected:  5538 Recovered:  124
Day # 12
Infected:  10636 Recovered:  281
Day # 13
Infected:  18120 Recovered:  599
Day # 14
Infected:  26156 Recovered:  1270
Day # 15
Infected:  31079 Re

,Day,Susceptible,Infected,Recovered,All Preventions,No Preventions,Only Hands,Only Mask,Only Social Dist,Hands and Mask,Hands and Social Dist,Mask and Social Dist
0,0,40001,1,0,0,1,0,0,4894,0,0,0
1,1,40000,2,0,0,2,0,0,4894,0,0,0
2,2,39997,5,0,1,3,0,1,4894,0,0,0
3,3,39992,10,0,1,5,1,1,4894,1,0,1
4,4,39977,25,0,3,8,5,4,4892,1,0,2
5,5,39948,53,1,8,12,7,11,4890,1,3,7
6,6,39878,122,2,13,26,15,17,4877,11,14,9
7,7,39721,276,5,20,55,39,33,4851,26,36,24
8,8,39403,589,10,42,126,79,81,4816,53,76,54
9,9,38732,1245,25,97,257,162,155,4713,106,171,118
